# About this Jupyter Notebook

@author: Yingding Wang

This notebook demonstrates fetching image from a private container registry from a Python KF pipeline SDK

In [32]:
import sys

In [22]:
# Debugging the environment variables
# %env

# Install kfp to build a pipeline
* Build KF pipeline with SDK: https://www.kubeflow.org/docs/components/pipelines/sdk/build-pipeline/
* Current KFP SDK version on pypi.org: https://pypi.org/project/kfp/ 

In [ ]:
!{sys.executable} -m pip install --upgrade --user kfp==1.8.11
# !{sys.executable} -m pip install --upgrade --user kubernetes==18.20.0
# !{sys.executable} -m pip install --upgrade --user kubernetes==21.7.0

In [34]:
!{sys.executable} -m pip show kubernetes

Name: kubernetes
Version: 18.20.0
Summary: Kubernetes python client
Home-page: https://github.com/kubernetes-client/python
Author: Kubernetes
Author-email: 
License: Apache License Version 2.0
Location: /home/jovyan/.local/lib/python3.8/site-packages
Requires: requests-oauthlib, setuptools, six, google-auth, requests, certifi, urllib3, pyyaml, websocket-client, python-dateutil
Required-by: kfp, kfserving


Name: kubernetes
Version: 12.0.1
Summary: Kubernetes python client
Home-page: https://github.com/kubernetes-client/python
Author: Kubernetes
Author-email: 
License: Apache License Version 2.0
Location: /opt/conda/lib/python3.8/site-packages
Requires: pyyaml, requests-oauthlib, setuptools, certifi, six, urllib3, websocket-client, google-auth, requests, python-dateutil
Required-by: kfp, kfserving

## Restart the Kernal

In [35]:
from platform import python_version
print (f"current platform python version: {python_version()}")

current platform python version: 3.8.10


In [ ]:
# run kubectl command line to see the quota in the name space
!kubectl describe quota

## Create K8s secret
* k8s python client API doc: https://github.com/kubernetes-client/python

In [37]:
from kubernetes import client as k8s_client
from kubernetes import config as k8s_config

K8_NAME_SPACE = 'kubeflow-kindfor'
K8_GIT_SECRET_NAME = 'lrzgit-kindfor-playground'

### Construct an empty k8s V1Object Reference

Use the following line to construct an empty K8s V1Object for KFP Python SDK.
```python
secret = k8s_client.V1ObjectReference(name=K8_GIT_SECRET_NAME, namespace=K8_NAME_SPACE)
```

In [38]:
secret = k8s_client.V1ObjectReference(name=K8_GIT_SECRET_NAME, namespace=K8_NAME_SPACE)

print(type(secret))
print(secret)
print(secret.namespace)

<class 'kubernetes.client.models.v1_object_reference.V1ObjectReference'>
{'api_version': None,
 'field_path': None,
 'kind': None,
 'name': 'lrzgit-kindfor-playground',
 'namespace': 'kubeflow-kindfor',
 'resource_version': None,
 'uid': None}
kubeflow-kindfor


### create a secret with kubectl

1. Create a Deployment Token on GIT LAB with an optional user name, and readonly_registory to pull image
2. SSH Login to your k8s cluster and manually create a docker-registry secret with
```console
kubectl -n kubeflow-kindfor create secret docker-registry <K8_GIT_SECRET_NAME> --docker-server=<GIT_URI> --docker-email=<an_abitrary_email> --docker-password=<deployment_token_string> --docker-username=<deployment_token_user_name>
```

Reference:
* https://kubernetes.io/docs/tasks/configure-pod-container/pull-image-private-registry/

Notice:\
use the following command to examin the docker-registry secret you created in terminal.
```console
kubectl -n <kubeflow_namespace> get secret <K8_GIT_SECRET_NAME> --output="jsonpath={.data.\.dockerconfigjson}" | base64 --decode
```

## Getting started with KubeFlow Python SDK

* Python SDK V2: https://www.kubeflow.org/docs/components/pipelines/sdk-v2/python-function-components/
* Python SDK Overview: https://www.kubeflow.org/docs/components/pipelines/sdk/sdk-overview/

In [39]:
EXPERIMENT_NAME = 'test containers pulling'        # Name of the experiment groups runs in the GUI
EXPERIMENT_DESC = 'Test pulling container with kfp SDK from a private image registory'
use_custom_image = True
#use_custom_image = False
if (use_custom_image):
    BASE_IMAGE = f"gitlab.lrz.de:5005/dzkj/playground:branch-docker-cd"
else:
    BASE_IMAGE = f"library/python:{python_version()}"
print(f"BASE_IMAGE: {BASE_IMAGE}")

BASE_IMAGE: gitlab.lrz.de:5005/dzkj/playground:branch-docker-cd


In [ ]:
import kfp
import kfp.dsl as dsl
# import kfp.v2.dsl as dsl
'''
from kfp.v2.dsl import (
    component,
    Input,
    Output,
    Dataset,
    Metrics
)
'''

In [44]:
def nothing():
    pass

def add(a: float, b: float) -> float:
    '''Calculates sum of two arguments'''
    print(a, '+', b, '=', a + b)
    return a + b

custom_image_op = kfp.components.create_component_from_func(
    func=add,
    output_component_file='playground_component.yaml', # This is optional. It saves the component spec for future use.
    base_image=BASE_IMAGE,
    #packages_to_install=['']
)

custom_image_op_nothing = kfp.components.create_component_from_func(
    func=nothing,
    output_component_file='playground_component.yaml', # This is optional. It saves the component spec for future use.
    base_image=BASE_IMAGE,
    #packages_to_install=['']
)

In [45]:
@dsl.pipeline(
    name = EXPERIMENT_NAME,
    description = EXPERIMENT_DESC
)
def custom_image_pipeline(
    a: float = 0,
    b: float = 7
):
    first_task = custom_image_op(a, b)
    

@dsl.pipeline(
    name = EXPERIMENT_NAME,
    description = EXPERIMENT_DESC
)
def custom_image_pipeline_nothing():
    first_task = custom_image_op_nothing()    
    

### Setting imagePullSecretes in K8s with Pipeline config
* Setting imagePullSecretes for Pipeline with SDK: https://github.com/kubeflow/pipelines/issues/5843#issuecomment-859799181

In [47]:
# from kubernetes import client as k8s_client
pipeline_config = kfp.dsl.PipelineConf()
if (use_custom_image):
    pipeline_config.set_image_pull_secrets([k8s_client.V1ObjectReference(name=K8_GIT_SECRET_NAME, namespace=K8_NAME_SPACE)])
    pipeline_config.set_image_pull_policy("IfNotPresent")
    # pipeline_config.set_image_pull_policy("Always")
arguments = {'a': '7', 'b': '8'}
arguments_nothing = {}

In [48]:
client = kfp.Client()
'''
client.create_run_from_pipeline_func(
    pipeline_func=custom_image_pipeline,
    arguments = arguments_nothing,
    pipeline_conf=pipeline_config,
    experiment_name=EXPERIMENT_NAME,
    namespace=K8_NAME_SPACE,
    # mode=kfp.dsl.PipelineExecutionMode.V2_COMPATIBLE # can not run wiht V2_COMPATIBLE
)
'''

client.create_run_from_pipeline_func(
    pipeline_func=custom_image_pipeline_nothing,
    arguments = arguments_nothing,
    pipeline_conf=pipeline_config,
    experiment_name=EXPERIMENT_NAME,
    namespace=K8_NAME_SPACE,
    # mode=kfp.dsl.PipelineExecutionMode.V2_COMPATIBLE # can not run wiht V2_COMPATIBLE
)

RunPipelineResult(run_id=d9ae0579-7982-42a1-b382-932ab1d2be18)